$\textbf{BNF Notation:}$

\begin{eqnarray}
  \textit{program} & ::= & \textit{statement} & \\
    & | & \textit{statement} \, \textit{program} & \\
    \\
  \textit{statement} & ::= & \textit{print} \, \textit{formula}; & \\
    & | & \textit{assign} \, \textit{formula}; & \\
    & | & \textit{return} \, \textit{formula}; & \\
    & | & \textit{import} \, \textit{formula}; & \\
    & | & \textit{skip}; \\
    \\
  \textit{formula} & ::= & \textit{False} & \\
    & | & \textit{True} & \\
    & | & \textit{not} \, ( \, \textit{formula} \, , \, \textit{formula} \, ) & \\
    & | & \textit{and} \, ( \, \textit{formula} \, , \, \textit{formula} \, ) & \\
    & | & \textit{or} \, ( \, \textit{formula} \, ) & \\
    & | & \textit{add} \, (\, \textit{formula} \, , \, \textit{formula} \, ) & \\
    & | & \textit{sub} \, (\, \textit{formula} \, , \, \textit{formula} \, ) & \\
    & | & \textit{mul} \, (\, \textit{formula} \, , \, \textit{formula} \, ) & \\
    & | & \textit{div} \, (\, \textit{formula} \, , \, \textit{formula} \,) & \\
        \textit{List} & :: = & \textit{number} & \\
    & | & (\textit{List} \, , \, \textit{number} \, ) & \\
\end{eqnarray}

In [139]:
import ast
import numpy as np
import inspect 

keys = {'ast','np','inspect'}
class evaluate_with_environment(ast.NodeVisitor):
    
    def __init__(self,verbose=True,keys={}):
        self.keywords = {'print'}.union(keys)
        self.verbose = verbose
        self.env = {}
        
    def help(self,key,val):
        if key in self.env:
            pass
        else:
            self.env[key] = val
            
    def visit_Module(self, node):
        if self.verbose:
            print(type(node))
        results = [self.visit(s) for s in node.body]
        if self.verbose:
            print()
            print('Results: ' + str(results))
            print('Enviroment: '+ str(self.env))
        return all(results)

    def visit_Assign(self, node):
        if self.verbose:
            print(type(node))
        self.help(node.targets[0].id,True)
        self.visit(node.targets[0])
        return self.visit(node.value)
    
    def visit_Expr(self, node):
        if self.verbose:
            print(type(node))
        return self.visit(node.value)

    def visit_BinOp(self, node):
        if self.verbose:
            print(type(node))
            return self.visit(node.value)

    def visit_Num(self, node):
        if self.verbose:
            print(str(type(node)) + ' : ' + str(node.n))
        return true 

    def visit_Name(self, node):
        name = node.id
        if self.verbose:
            print(str(type(node)) + ' : ' + name)
        if name in self.keywords or name in self.env:
            return True
    
    def visit_FunctionDef(self,node):
        if self.verbose:
            print(str(type(node)) + ' : ' +node.name + str([arg.arg for arg in node.args.args])) 
        self.help(node.name,True)
        results = [self.visit(s) for s in node.body]
        if self.verbose:
            print(type(node))
        return all(results)
    
    def visit_Call(self,node):
        if self.verbose:
            print(type(node))
        return self.visit(node.func) and all([self.visit(arg) for arg in node.args])
    
    def visit_Str(self,node):
        if self.verbose:
            print(type(node))
        return True
    
    def visit_Import(self,node):
        if self.verbose:
            print(str(type(node)) + " : " + str([alias.asname for alias in node.names])) 
        self.help(node.names[0].asname,node.names[0].name)
        return True
    
    def visit_Attribute(self,node):
        if self.verbose:
            print(str(type(node)) + ' : ' + node.attr)
        return self.visit(node.value)
    
    def visit_Return(self,node):
        if self.verbose:
            print(type(node))
        return self.visit(node.value)
    
    def visit_List(self,node):
        if self.verbose:
            print(type(node))
        return [self.visit(x) for x in node.elts]



def ex_1():
    x = np.ndarray([1,2,3],[4,5,6])
    y = np.ndarray([7,8,9],[3,2,1])
    z = np.ndarray([4,5,6],[1,2,3])
    a = np.add(x,y)
    return np.add(z,a)

def ex_2():
    x = np.ndarray([1,2,3],[4,5,6])
    y = np.ndarray([7,8,9],[3,2,1])
    return np.subtract(x,y)

source = inspect.getsource(ex_2)
a_s_t = ast.parse(source)
print('\nSource: \n' + source + '\n')
print('Abstract Syntax Tree:\n' + str(a_s_t) + '\n')
print()
print(evaluate_with_environment().visit(a_s_t))

# source = inspect.getsource(ex_2)
# a_s_t_2 = ast.parse(source)
# print('\nSource: \n' + source + '\n')
# print('Abstract Syntax Tree:\n' + str(a_s_t_2) + '\n')
# print()
# print(valid_1(a_s_t_2))

#ast.dump(test_node)
#print(evaluate_with_environment().visit_Module(test_node))
#print(evaluate_with_environment().visit_Num(test_node))








Source: 
def ex_2():
    x = np.ndarray([1,2,3],[4,5,6])
    y = np.ndarray([7,8,9],[3,2,1])
    return np.subtract(x,y)


Abstract Syntax Tree:


<class '_ast.Module'>
<class '_ast.FunctionDef'> : ex_2[]
<class '_ast.Assign'>
<class '_ast.Name'> : x
<class '_ast.Call'>
<class '_ast.Attribute'> : ndarray
<class '_ast.Name'> : np
<class '_ast.Assign'>
<class '_ast.Name'> : y
<class '_ast.Call'>
<class '_ast.Attribute'> : ndarray
<class '_ast.Name'> : np
<class '_ast.Return'>
<class '_ast.Call'>
<class '_ast.Attribute'> : subtract
<class '_ast.Name'> : np
<class '_ast.FunctionDef'>

Results: [False]
Enviroment: {'ex_2': True, 'x': True, 'y': True}
False
